In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import os
import json
import cv2
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from collections import defaultdict
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog

In [ ]:
# Register a COCO format dataset to Detectron2
register_coco_instances("train_seed", {}, 
    "/kaggle/input/superai-north-seed-detection/boundarybox_seed_train.json", 
    "/kaggle/input/superai-north-seed-detection/train")

seed_metadata = MetadataCatalog.get("train_seed")

In [ ]:
# Set up training configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('LVISv0.5-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_1x.yaml'))
cfg.DATASETS.TRAIN = ("train_seed",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('LVISv0.5-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_1x.yaml')  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 20  # you have 20 classes of seeds

In [ ]:
# Train the model
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Set up prediction configuration
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
# cfg.DATASETS.TEST = ("seed_train", )
predictor = DefaultPredictor(cfg)

# Submission

In [ ]:
def auto_crop(image_path):
    # Load image
    img = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (5,5), 0)

    # Threshold the image
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours in the binary image
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Find bounding rectangles for each contour
    rects = [cv2.boundingRect(cnt) for cnt in contours]

    # Find the combined bounding box
    top_x = min([x for (x, y, w, h) in rects])
    top_y = min([y for (x, y, w, h) in rects])
    bottom_x = max([x+w for (x, y, w, h) in rects])
    bottom_y = max([y+h for (x, y, w, h) in rects])

    # Crop the image with the dimensions of the bounding box
    cropped = img[top_y:bottom_y, top_x:bottom_x]

    # Resize the image
    resized = cv2.resize(cropped, (3648, 2736))

    return resized

# Use the function to auto-crop an image
cropped_img = auto_crop('/kaggle/input/superai-north-seed-detection/test/150.jpg')

# Show the cropped image
plt.imshow(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# Directory containing images to be cropped
source_dir = '/kaggle/input/superai-north-seed-detection/test'

# Directory to save cropped images
dest_dir = './cropped_img'

# Create the destination directory if it does not exist
os.makedirs(dest_dir, exist_ok=True)

# Iterate over all images in the source directory
for filename in os.listdir(source_dir):
    # Ensure the file is an image
    if filename.endswith(".jpg"):
        # Construct full image path
        image_path = os.path.join(source_dir, filename)
        
        # Auto crop the image
        cropped_img = auto_crop(image_path)
        
        # Save the cropped image to the destination directory
        cv2.imwrite(os.path.join(dest_dir, "" + filename), cropped_img)

In [ ]:
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# select an image file
filename = "105.jpg"   # replace with your image file name
img_path = os.path.join("/kaggle/working/cropped_img/", filename)

# load the image with OpenCV
img = cv2.imread(img_path)

# make a prediction with the model
outputss = predictor(img)

# create a Visualizer instance
v = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get("seed_train"), scale=1.2)

# overlay the predictions on the image
v = v.draw_instance_predictions(outputss["instances"].to("cpu"))

# display the image
plt.imshow(v.get_image()[:, :, ::-1])
plt.show()

In [ ]:
# location of submission data
submission_dir = '/kaggle/working/cropped_img'
# get the list of submission image files
sub_files = os.listdir(submission_dir)
print(sub_files)
print(len(sub_files))

In [ ]:
# initialize dictionary to hold counts
counts = defaultdict(lambda: [0]*20)

for filename in sub_files:
    if filename.endswith(".jpg"):   # make sure we're working with an image
        # read the image file
        img = cv2.imread(os.path.join(submission_dir, filename))
        
        # use the predictor on the image
        outputs = predictor(img)
        
        # get the predicted classes for each instance
        classes = outputs["instances"].pred_classes.to("cpu").numpy()

        # increment the count for each class
        for class_id in classes:
            counts[filename][class_id] += 1
        
        # if no classes were found, ensure the image still gets added to the dictionary with zero counts
        if filename not in counts:
            counts[filename] = [0]*20

In [ ]:
# convert counts to a dataframe
df = pd.DataFrame(counts).T

# if any images had no instances of a class, fill that with 0
df.fillna(0, inplace=True)

# name columns with class names
df.columns = seed_metadata.thing_classes
df.index.name = 'id'
df

In [ ]:
# write the dataframe to a csv file
df.to_csv('seed08.csv')